<a href="https://colab.research.google.com/github/hurgj123kr/Project/blob/main/%EB%8B%A4%EC%9D%8C%EB%B6%84%EA%B8%B0_%EA%B2%8C%EC%9E%84_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  다음분기의 게임 데이터셋을 불러옵니다.
### 1)결측치가 있다면 제거해주고
### 2)연도별로 정렬해줍니다.
### 3)각 지역별 출고값열들 특정 문자 제거후 타입변경
#### 4)이상한 연도들 원래 연도값으로 변경해주기

In [ ]:
import pandas as pd
import numpy as np
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
url ='https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
df = pd.read_csv(url,thousands=',')
#데이터 확인
df.columns
df.shape
#데이터에 이상이없는지 확인합니다
df.isnull().sum()
##데이터에 문제가있으므로 제거해줍니다.
df.dropna(inplace=True)
df.isnull().sum()
#이제 연도별로 정렬해줍니다.
df.sort_values(by=['Year'], axis=0,inplace=True)
df.drop(['Unnamed: 0'], axis=1,inplace=True)
#df = df.replace({'K':'', 'M':''}, regex=True)
#df = df.astype({'NA_Sales':'float','EU_Sales':'float','JP_Sales':'float',
#                          'Other_Sales':'float'})
df.dtypes
df

In [ ]:
#연도 이상치 고치기
year = df['Year'].unique()
temp = []
for i in year:
    if i <= 1900:
        print(i)
        temp.append(i)

df.loc[(df['Year'].values <= 80),'Year' ] += 2000
df.loc[((df['Year'].values >=80) & (df['Year'].values <= 100)),'Year'] += 1900
df['Year'].value_counts()
df

In [ ]:
df.NA_Sales.value_counts()

# 이제 불러온 데이터를 통해서 지역에따라 장르의 차이를 나타낸 표그리기


**밑의 막대그래프에따르면 북미지역은 액션게임 출고량이높고,유럽지역도 액션게임출고량이높습니다.**


**일본지역 Role-Playing게임이 출고량이높네요, 기타지역은 액션게임의 출고량이 높습니다.**


**그러므로 북미,유렵,기타지역은 액션게임을 선호합니다.**


**일본만 Role-playing게임을 선호합니다**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#각 장르별 나라들의 출고값을 합산한것입니다.
game_genre = df[['Genre','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
game_genre.columns =['장르','북미지역 출고량','유렵지역 출고량','일본지역 출고량','기타지역 출고량']
genre_group = game_genre.groupby(['장르']).sum()
game_df = genre_group.reset_index()
#확인
game_df
game_graph = pd.melt(game_df, id_vars=['장르'], value_vars=game_df.columns[1:],
                      var_name='판매지역', value_name='출고량')
game_graph.head()
plt.figure(figsize=(14, 13))
sns.barplot(data=game_graph, x='장르', y='출고량', hue='판매지역')
plt.title('국가별 장르 선호도', loc='center', fontsize=24)
plt.legend(fontsize=15)
plt.show()